In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
# ! git pull

## Add import path

In [ ]:
import gc
import os
import sys

In [ ]:
def add_library_level(level=4):
    suf_path = ['..']
    path = '..'
    for i in range(0, level):
        join_path = suf_path * i
        path = '/'.join(join_path)
        module_path = os.path.abspath(os.path.join(path))
        if module_path not in sys.path:
            sys.path.append(module_path)
            print(f'Appendeding {path}')

In [ ]:
add_library_level(level=4)

## Organize imports

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from torch import nn

In [ ]:
from sae_lens import SAE, HookedSAETransformer
from transformer_lens.utils import tokenize_and_concatenate

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
from datasets import load_dataset

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from tqdm import tqdm

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
import plotly.express as px

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

In [ ]:
from src.lattmc.sae.nlp_sae_utils import *

In [ ]:
import logging

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
device = init_device()

In [ ]:
torch.__version__

In [ ]:
np.__version__

## Initialize Layer

In [ ]:
layer = 8
dataset_suffix = 'pile'

## Initialize Path

In [ ]:
def init_data(model_name, layer, dataset_suffix='pile'):
    PATH = Path('../data')
    dataset_suffix = 'pile'
    vectors_name = f'{model_name.replace('-', '_')}_{layer}_{dataset_suffix}'
    checkpoint_dir = PATH / 'saes'
    vectors_dir = checkpoint_dir / f'{vectors_name}_vecs'
    matrix_dir = checkpoint_dir / f'{vectors_name}_mats'
    vectors_path = checkpoint_dir / f'{vectors_name}_vecs.joblib'
    
    mkdirs(
        checkpoint_dir,
        vectors_dir,
        matrix_dir
    )
    logger.info(f'{matrix_dir = } {vectors_path = }')

    return matrix_dir, vectors_path

## Initialize simple dataset

In [ ]:
dataset = load_dataset(
    path='NeelNanda/pile-10k',
    split='train',
    streaming=False,
)

## Initialize model

In [ ]:
def gen_concept(idx, val, shape=24576):
    v_idx = np.zeros((shape,), dtype=float)
    v_idx[idx] = val
    concept = fca.G_FG(v_idx)
    
    return v_idx, concept

In [ ]:
model_name = 'gpt2-small'
release = 'gpt2-small-res-jb'
sae_id = f'blocks.{layer}.hook_resid_pre'

In [ ]:
matrix_dir, vector_path = init_data(model_name, layer, dataset_suffix=dataset_suffix)

In [ ]:
net = Text2Sae(
    model_name,
    release,
    sae_id,
    device
)

## Generate V Lattice

In [ ]:
gc.collect()

In [ ]:
matrix_dir

In [ ]:
init_matrices(matrix_dir, dataset, net)

In [ ]:
gc.collect()

In [ ]:
V = init_vectors(vectors_path, matrix_dir, segment=False)

## Generate Context and Analyze

In [ ]:
gc.collect()

In [ ]:
fca = FCA(V)

In [ ]:
text1 = "The Golden Gate Bridge"
text2 = "The Brooklyn Bridge"
text3 = "The card game Bridge"

In [ ]:
u_1s = net.embed(text1)[0][1:]
u_2s = net.embed(text2)[0][1:]
u_3s = net.embed(text3)[0][1:]
u_1s.shape, u_2s.shape, u_2s.shape

In [ ]:
u_1b = u_1s[-1]
u_2b = u_2s[-1]
u_3b = u_3s[-1]

In [ ]:
m_s = meet(u_1b, u_2b)
topK(m_s, 20)

In [ ]:
m_s = meet(u_1b, u_3b)
topK(m_s, 20)

In [ ]:
m_s = meet(u_2b, u_3b)
topK(m_s, 20)

In [ ]:
m_s = meet(u_2b, u_3b)
m_s = meet(u_1b, m_s)
topK(m_s, 20)

In [ ]:
c_1b = fca.G_FG(u_1b)
c_2b = fca.G_FG(u_3b)
c_3b = fca.G_FG(u_3b)
c_1b, c_2b, c_3b

In [ ]:
c_mb = fca.G_FG(m_s)
c_mb

In [ ]:
c_mb.A[0]

In [ ]:
v_d = V[188]

In [ ]:
topK(v_d, 100)

In [ ]:
le(m_s, v_d)

In [ ]:
_, c_d = gen_concept(1030, 80)
c_d

In [ ]:
c_d.A

In [ ]:
vs = joblib.load(matrix_dir / f'{c_d.A[0]}.joblib')

In [ ]:
vs.toarray()

In [ ]:
def select_ws(A, v):
    idxs = []
    for i in A:
        v_sparse = joblib.load(matrix_dir / f'{i}.joblib')
        print(matrix_dir / f'{i}.joblib')
        vs = v_sparse.toarray()
        gs = find_G_x(vs, v)
        idxs.append(gs)

    return idxs

In [ ]:
select_ws([c_d.A[0]], m_s)

In [ ]:
V.shape

In [ ]:
idx = c_d.A[0].item()

In [ ]:
v_sparse = joblib.load(matrix_dir / f'{idx}.joblib')
vs = v_sparse.toarray()
gs = find_G_x(vs, m_s)

In [ ]:
v_max = join_all(vs[1:])
vs.shape, v_max.shape

In [ ]:
fca.V.shape

In [ ]:
vs.shape

In [ ]:
dataset[idx]

In [ ]:
tokens = net.tokenize(dataset[idx]['text'])[0]

In [ ]:
tokens.shape

In [ ]:
np.where(v_max != V[c_d.A[0]])

In [ ]:
np.all(v_max == fca.V[c_d.A[0]])

In [ ]:
c_d = fca.G_FG(m_s)

In [ ]:
le(m_s, v_max), le(m_s, fca.V[c_d.A[0]]), 

In [ ]:
gs

In [ ]:
w_s = select_ws([idx], m_s)

In [ ]:
v_s, c_s = gen_concept(17608, 38)

In [ ]:
idx = c_s.A[0].item()

In [ ]:
v_sparse = joblib.load(matrix_dir / f'{idx}.joblib')
vs = v_sparse.toarray()
gs = find_G_x(vs, m_s)

In [ ]:
v_max = join_all(vs[1:])

In [ ]:
for i, v in enumerate(vs):
    if le(v_s, v):
        print(le(v_s, v), i)

In [ ]:
tokens = net.tokenize(dataset[idx]['text'])[0]
t_s = tokens[i]
t_s

In [ ]:
tokens.shape, vs.shape

In [ ]:
net.to_string(t_s)

In [ ]:
np.where(m_s > 0)

In [ ]:
le(m_s, join_all(vs[1:]))

In [ ]:
w_s

In [ ]:
dataset[174]

In [ ]:
fca.G_FG(v_d)

In [ ]:
dataset[188]

In [ ]:
dataset[c_mb.A]

In [ ]:
z = net.encode(text1)
zs = z.to('cpu').detach().numpy()[0]
v = np.maximum.reduce(zs)

In [ ]:
zs[1:].shape

In [ ]:
zs.shape

In [ ]:
v[19837]

In [ ]:
v_g = net.embed('Golden')[0][1]
v_t = net.embed('Gate')[0][1]
v_b = net.embed('Bridge')[0][1]

In [ ]:
us = net.embed(text1)[0]
us.shape

In [ ]:
us = us[1:]
us.shape

In [ ]:
u_g = us[1]
u_t = us[2]
u_b = us[3]

In [ ]:
np.argmax(u_g), np.argmax(v_g)

In [ ]:
topK(u_g, 10), topK(v_g, 10)

In [ ]:
ws = net.embed("The Brooklyn Bridge")[0]
ws.shape

In [ ]:
net.tokenize(text1)[0]

In [ ]:
net.tokenize("The Brooklyn Bridge")[0]

In [ ]:
ws = us[1:]
ws.shape

In [ ]:
w_c = us[1]
w_b = us[2]

In [ ]:
topK(u_b, 10), topK(w_b, 10)

In [ ]:
m_b = meet(u_b, w_b)
topK(m_b, 20)

In [ ]:
m_b = meet(w_b, v_b)
topK(m_b, 20)

In [ ]:
v_g = net.embed('Golden')[0][1]
v_t = net.embed('Gate')[0][1]
v_b = net.embed('Bridge')[0][1]

In [ ]:
v_g.shape

In [ ]:
c_g = fca.G_FG(v_g)
c_t = fca.G_FG(v_t)
c_b = fca.G_FG(v_b)

In [ ]:
c_g10 = fca.G_FG(v_g / 10)

In [ ]:
texts = dataset[c_g10.A]

In [ ]:
texts['text'][0]

In [ ]:
topK(v_g, 22)

In [ ]:
v_g = net.embed(' golden')[0][1:]
t_g = net.tokenize(' golden')[0][1:]

In [ ]:
v_g.shape, t_g.shape

In [ ]:
net.to_string(t_g[0])

In [ ]:
t_g

In [ ]:
net.tokenize('Golden')[0]

In [ ]:
net.tokenize(' golden')[0]

In [ ]:
net.tokenize('golden')[0]

In [ ]:
topK(v_g[0], 20)

In [ ]:
v_g, c_g = gen_concept(10261, 20)
c_g

In [ ]:
tk = net.tokenize(dataset[c_g.A[0].item()]['text'])[0]

In [ ]:
ids = select_ws([c_g.A[0].item()], v_g)
ids

In [ ]:
tk.shape

In [ ]:
for it in ids[0]:
    print(net.to_string(tk[it]))

In [ ]:
fca.G_FG(net.embed(' golden')[0])

## New York Ciry

In [ ]:
v_ns = net.embed('New York City')[0][1:]
v_gs = net.embed('Golden Gate Bridge')[0][1:]

In [ ]:
t_ns = net.tokenize('New York City')[0][1:]
t_gs = net.tokenize('Golden Gate Bridge')[0][1:]

In [ ]:
t_ns.shape, t_gs.shape

In [ ]:
t_ns, t_gs

In [ ]:
m_bg = meet(v_ns[2], v_gs[2])
topK(m_bg, 20)

In [ ]:
m_cg, c_bg = gen_concept(19016, 5)
c_bg

In [ ]:
topK(c_bg.v, 20)

In [ ]:
c_bg = fca.G_FG(m_bg)
c_bg

In [ ]:
tk = net.tokenize(dataset[c_bg.A[2].item()]['text'])[0]

In [ ]:
ids = select_ws([c_bg.A[2].item()], m_cg)
ids

In [ ]:
tk.shape

In [ ]:
for it in ids[0]:
    print(net.to_string(tk[it]))

In [ ]:
phrases = [
    "Golden Gate Bridge",
    "Golden Gate",
    "Gate",
    "Bridge",
    "Golden",
    "New York City",
    "New York",
    "City",
    "New",
    "hot dog",
    "hot",
    "dog",
]

In [ ]:
vecs = net.embed(phrases)

In [ ]:
vecs.shape

In [ ]:
phrases[4]

In [ ]:
vec = vecs[4]

In [ ]:
tokens = net.tokenize(phrases[4])[0]

In [ ]:
tokens.shape

In [ ]:
net.to_string(tokens[1])

In [ ]:
vecs1 = net.embed('Golden')[0]
vecs2 = net.embed('Gold')[0]
vecs3 = net.embed('Golden Gate Bridge')[0]

In [ ]:
np.allclose(vec[1], vecs3[1])

In [ ]:
np.all(vecs1[1] == vecs3[1])

In [ ]:
np.all(vec[1] == vecs2[1])

In [ ]:
topK(vecs2[1], 32)

In [ ]:
topK(vec[1], 32)

In [ ]:
for text in phrases:
    h = net.embed(text)              # encode to sparse feature space
    active_feats = np.where(h > 0)[0]       # indices of active features
    top_feats = active_feats[np.argsort(-h[active_feats])[:5]]  # top 5 features by magnitude
    print(f"\nInput: '{text}'")
    print(" Top active feature indices:", top_feats)

## Golden Feature from Neuronscope

In [ ]:
v_golden, concept = gen_concept(19837, 90)
v_golden.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Cat Feature from Neuronscope

In [ ]:
v_cat, concept = gen_concept(16899, 80)
v_cat.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Apple Feature from Neuronscope

In [ ]:
v_apple, concept = gen_concept(4269, 70)
v_apple.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Thunder and Lightning Feature from Neuronscope

In [ ]:
v_thunder, concept = gen_concept(23123, 40)
v_thunder.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## School and Lightning Feature from Neuronscope

In [ ]:
v_school, concept = gen_concept(20781, 84)
v_school.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## King and Lightning Feature from Neuronscope

In [ ]:
v_king, concept = gen_concept(17624, 60)
v_king.shape, concept

In [ ]:
dataset[concept.A[1].item()]

## Orgs and Lightning Feature from Neuronscope

In [ ]:
v_orgs, concept = gen_concept(16660, 60)
v_orgs.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## States and Lightning Feature from Neuronscope

In [ ]:
v_state, concept = gen_concept(22, 10)
v_state.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Joint concepts

In [ ]:
# topK(v_orgs, 4), topK(v_school, 4)

In [ ]:
v_orgs = np.zeros_like(to_numpy(v))
v_school = np.zeros_like(to_numpy(v))
v_having = np.zeros_like(to_numpy(v))

In [ ]:
v_orgs[16660] = 20
v_school[17624] = 20
v_having[17935] = 6

In [ ]:
orgs = fca.G_FG(v_orgs)
schools = fca.G_FG(v_school)
havings = fca.G_FG(v_having)
orgs, schools, havings

In [ ]:
org_school = orgs & schools
org_school

In [ ]:
dataset[org_school.A[0].item()]['text']

In [ ]:
org_school_havings = orgs & schools & havings 
org_school_havings

In [ ]:
dataset[org_school_havings.A[0].item()]['text']

## Recipie Feature from Neuronscope

In [ ]:
v_recipie, concept = gen_concept(7, 28)
v_recipie.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Election Feature from Neuronscope

In [ ]:
v_elect, concept = gen_concept(29, 28)
v_elect.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Mixture of Election and Party

In [ ]:
v_elect, concept_elects = gen_concept(29, 2)
v_elect.shape, concept

## Food Features

In [ ]:
texts = [
    'food recipie',
    'love',
    'admire',
    'sex',
]

In [ ]:
v = net.encode(texts[2])[0][1]
concept = fca.G_FG(v / 4)
concept

In [ ]:
dataset[concept.A[3].item()]

In [ ]:
topK(v, 10)

In [ ]:
v_love, recipie = gen_concept(14654, 20)
v_love.shape, concept

In [ ]:
v_admire, recipie = gen_concept(14990, 20)
v_admire.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Verbal Feature from Neuronscope

In [ ]:
v_verbal, concept = gen_concept(33, 40)
v_verbal.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Border Feature from Neuronscope

In [ ]:
v_border, concept = gen_concept(35, 20)
v_border.shape, concept

In [ ]:
dataset[concept.A[0].item()]

## Cross Concept

In [ ]:
v_verbal, concept_verbal = gen_concept(33, 16)
v_border, concept_border = gen_concept(35, 16)

In [ ]:
cross = concept_verbal & concept_border
cross

In [ ]:
dataset[concept.A[1].item()]

## Cross Concept Probability

In [ ]:
v_posib, concept_posib = gen_concept(1061, 20)
v_posit, concept_posit = gen_concept(809, 20)
v_liklh, concept_liklh = gen_concept(418, 20)
v_qualt, concept_qualt = gen_concept(129, 20)

In [ ]:
cross_pr_ps = concept_posib & concept_posit
cross_pr_ps

In [ ]:
dataset[cross_pr_ps.A[0].item()]

In [ ]:
cross_lk_ql = concept_liklh & concept_qualt
cross_lk_ql

In [ ]:
dataset[cross_lk_ql.A[0].item()]

## Context Features

In [ ]:
v_peval, concept_peval = gen_concept(3109, 20)
v_evstt, concept_evstt = gen_concept(3784, 20)
v_sosis, concept_sosis = gen_concept(3702, 20)
v_agevs, concept_agevs = gen_concept(3388, 20)

In [ ]:
concept_pe_ev = concept_peval & concept_evstt
concept_pe_ev

In [ ]:
dataset[concept_pe_ev.A[0].item()]

In [ ]:
concept_so_ag = concept_sosis & concept_agevs
concept_so_ag

In [ ]:
dataset[concept_so_ag.A[0].item()]

In [ ]:
concept_all = concept_peval & concept_evstt & concept_sosis & concept_agevs
concept_all

In [ ]:
dataset[concept_all.A[0].item()]

## Computer and Technilogies Concepts

In [ ]:
v_comps, concept_comps = gen_concept(20542, 14)
v_comps.shape, concept_comps

In [ ]:
dataset[concept_comps.A[0].item()]

## Innovation Features

In [ ]:
v_innov, concept_innov = gen_concept(2503, 10)
v_innov.shape, concept_innov

In [ ]:
dataset[concept_innov.A[0].item()]

In [ ]:
concept_co_in = concept_comps & concept_innov
concept_co_in

In [ ]:
dataset[concept_co_in.A[0].item()]

## Add Examples

In [ ]:
text1 = " The Golden Gate Bridge"
z = net.encode(text1)
tokens = net.tokenize(text1)
print(torch.topk(z, 12))
print()
print(z)

In [ ]:
r = net.encode('Golden')
torch.topk(r[0][1], 50)

In [ ]:
torch.nonzero(r[0][1]).shape

In [ ]:
texts = [
    "Golden Gate Bridge",
    "New York City",
    "Silicon Valley",
    "The White House",
    "Apple Inc."
]

In [ ]:
texts = [
    "School of AI",
    "School of Economics",
    "School of Medicine",
    "School of Arts",
    "School of Technologies"
]

In [ ]:
texts = "Golden Retriever"
tokens = net.tokenize(texts)
tokens, net.to_string(tokens)

In [ ]:
texts = "Gold"
tokens = net.tokenize(texts)
vs = net.encode(texts)
tokens, net.to_string(tokens)

In [ ]:
net.to_string(32378)

In [ ]:
vs = net.encode(texts)

In [ ]:
vs.shape, vs[0].shape

In [ ]:
v_t = [v[1] for v in vs[:3]]
v_t[0].shape

In [ ]:
v_A = v_t[0]
for v in v_t:
    v_A = torch.minimum(v_A, v)

In [ ]:
vs[0][0].shape, 19837

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(vs[0][1], 20)

In [ ]:
torch.topk(v_A, 30)

## Golden Second Index

In [ ]:
v_1, concept_1 = gen_concept(21286, 16)
v_1.shape, concept_1

In [ ]:
dataset[concept_1.A[0].item()]

## Gold Second Index

In [ ]:
v_2, concept_2 = gen_concept(9572, 30)
v_2.shape, concept_2

In [ ]:
dataset[concept_2.A[0].item()]

In [ ]:
concept_m = concept_1 & concept_2
concept_m

In [ ]:
dataset[concept_m.A[1].item()]

In [ ]:
tk = topK(V[0], 30)
tk

In [ ]:
v_A = np.zeros_like(v_t[0].to('cpu').detach().numpy())
v_A[17943] = 62
v_A[11811] = 6
v_A[15823] = 4
v_A[4507] = 4
v_A[20161] = 4
concept = fca.G_FG(v_A)
concept

In [ ]:
concept.A.shape, V.shape, V.shape[0] - concept.A.shape[0]

In [ ]:
concept.A[0]

In [ ]:
dataset[concept.A[2].item()]

## Analyze Tokens

In [ ]:
len(dataset)

In [ ]:
checks = []
with tqdm(list(range(len(dataset)))) as pdata:
    for idx in pdata:
        tokens = net.tokenize(dataset[idx]['text'])[0]
        vs = joblib.load(matrix_dir / f'{idx}.joblib')
        checks.append(tokens.shape[0] == vs.shape[0])

In [ ]:
np.all(checks)

In [ ]:
from IPython.display import clear_output
v_paths = list(matrix_dir.glob('*.joblib'))
T_dict = {}
W_dict = {}
with tqdm(concept.A) as v_ppaths:
    for idx in v_ppaths:
        vs = joblib.load(matrix_dir / f'{idx}.joblib').toarray()
        G_x = find_G_x(vs, v_A)
        if G_x.shape[0] > 0:
            T_dict[idx.item()] = G_x
        clear_output(wait=True)

In [ ]:
len(T_dict)

In [ ]:
T_dict[5]

In [ ]:
tokens = net.tokenize(dataset[5]['text'])

In [ ]:
net.to_string(tokens[0][100])

In [ ]:
T_dict[5]

In [ ]:
net.to_string(tokens[0][0]), net.to_string(tokens[0][T_dict[5]])

In [ ]:
W_dict = {}
for k, v in T_dict.items():
    tokens = net.tokenize(dataset[k]['text'])[0]
    W_dict[k] = net.to_string(tokens[v])

In [ ]:
W_dict